In [1]:
import numpy as np
import pandas as pd

# 数据聚合和分组运算

**先说一下什么是分组运算，什么是聚合。一个表格型数据集，我们需要对某一标签下的数据进行运算，对于具有某种特性的数据进行运算。对于数据分析的角度，我们想到的就是将这个表格型数据集分隔--运算---合并**

## GroupBy机制

**正如上面我们讲的，我们将自己数据集分隔，然后运算，最后聚合。在pandas里面，有groupby可以完成，为了更好的理解分隔--运算--合并的过程，下面开始拿例子看看：**

In [2]:
frame = pd.DataFrame({'key1':['a','s','a','s','g'],'key2':['one','two','three','four','five'],'data1':range(5),'data2':np.random.randn(5)})
print(frame)

  key1   key2  data1     data2
0    a    one      0 -0.187529
1    s    two      1 -1.621045
2    a  three      2 -0.309963
3    s   four      3 -1.154808
4    g   five      4 -0.887752


In [3]:
#以key1分离数据
grouped = frame.groupby(frame['key1'])

In [4]:
for name,result in grouped:
    print('分离标签：')
    print(name)

    print('分离数据：')
    print(result)
    print('====================')

分离标签：
a
分离数据：
  key1   key2  data1     data2
0    a    one      0 -0.187529
2    a  three      2 -0.309963
分离标签：
g
分离数据：
  key1  key2  data1     data2
4    g  five      4 -0.887752
分离标签：
s
分离数据：
  key1  key2  data1     data2
1    s   two      1 -1.621045
3    s  four      3 -1.154808


In [5]:
#对数据做运算+合并
print(grouped.mean())

      data1     data2
key1                 
a         1 -0.248746
g         4 -0.887752
s         2 -1.387926


**上面使用grouped.mean()的运算步骤是：先分别算各自的mean，然后在做了合并**

**我们可以分析，groupby的so传入的是一个类list（可以是有索引关系的map和Series），我们通过传入的类list，将数据分隔，然后运算和合并，看下面例子**

In [6]:
state = ['lala','hahah','lala','mumumu','xixixi']
state

['lala', 'hahah', 'lala', 'mumumu', 'xixixi']

In [7]:
frame[['data1','data2']].groupby(state)

In [8]:
for name,result in frame[['data1','data2']].groupby(state):
    print('分离标签：')
    print(name)

    print('分离数据：')
    print(result)
    print('====================')

分离标签：
hahah
分离数据：
   data1     data2
1      1 -1.621045
分离标签：
lala
分离数据：
   data1     data2
0      0 -0.187529
2      2 -0.309963
分离标签：
mumumu
分离数据：
   data1     data2
3      3 -1.154808
分离标签：
xixixi
分离数据：
   data1     data2
4      4 -0.887752


**做一个总结：groupby里面传入的就是一个类list数据，它的机制是会根据你传入的类list的数据进行分隔，哪怕把这个数据之前不属于这个表格类型数据。要被运算的表格类型数据要传输的是它的可被运算数据，提取方式是DataFrame数据的提取方式，如果不进行选择，那么就默认为所有可运算的数据**

In [9]:
#再看一个例子
print(frame.groupby(frame['key1']).mean())

      data1     data2
key1                 
a         1 -0.248746
g         4 -0.887752
s         2 -1.387926


**上面的例子是选取了key1这个列，然后算的是所有数据的平均值，这时候结果自动屏蔽key2的元素。这个key2的元素就是所谓的【麻烦列】**

### 对分组迭代

**其实分组迭代我们在之前已经见过，就是使用for循环来将数据输出，但是里面还是会有一些其他的东西来做分享**

In [10]:
#例子1
for names,result in frame.groupby(['key1','key2']):  #多个元素的传入，不能写为frame[['key1','key2']]
    print('分离标签：')
    print(names)

    print('分离数据：')
    print(result)
    print('====================')

分离标签：
('a', 'one')
分离数据：
  key1 key2  data1     data2
0    a  one      0 -0.187529
分离标签：
('a', 'three')
分离数据：
  key1   key2  data1     data2
2    a  three      2 -0.309963
分离标签：
('g', 'five')
分离数据：
  key1  key2  data1     data2
4    g  five      4 -0.887752
分离标签：
('s', 'four')
分离数据：
  key1  key2  data1     data2
3    s  four      3 -1.154808
分离标签：
('s', 'two')
分离数据：
  key1 key2  data1     data2
1    s  two      1 -1.621045


**从多个维度分隔时，groupby传入的是带有标签的list（此时传入frame[['key1','key2']]会报错）**

In [11]:
#例子2
state = frame.dtypes
state

key1      object
key2      object
data1      int64
data2    float64
dtype: object

In [12]:
for name,result in frame.groupby(state,axis=1):
    print('分离标签：')
    print(name)

    print('分离数据：')
    print(result)
    print('====================')

分离标签：
int64
分离数据：
   data1
0      0
1      1
2      2
3      3
4      4
分离标签：
float64
分离数据：
      data2
0 -0.187529
1 -1.621045
2 -0.309963
3 -1.154808
4 -0.887752
分离标签：
object
分离数据：
  key1   key2
0    a    one
1    s    two
2    a  three
3    s   four
4    g   five


**我们一般是根据行数据的做分隔（默认axis=0），如果是用列数据做分隔，那么你需要使用axis=1,下面再做一个例子**

In [13]:
frame.iloc[1]

key1           s
key2         two
data1          1
data2   -1.62105
Name: 1, dtype: object

In [14]:
frame['key1']

0    a
1    s
2    a
3    s
4    g
Name: key1, dtype: object

In [15]:
for name,result in frame.groupby(frame.iloc[1],axis=1):
    print('分离标签：')
    print(name)

    print('分离数据：')
    print(result)
    print('====================')

分离标签：
-1.6210451710090494
分离数据：
      data2
0 -0.187529
1 -1.621045
2 -0.309963
3 -1.154808
4 -0.887752
分离标签：
1
分离数据：
   data1
0      0
1      1
2      2
3      3
4      4
分离标签：
s
分离数据：
  key1
0    a
1    s
2    a
3    s
4    g
分离标签：
two
分离数据：
    key2
0    one
1    two
2  three
3   four
4   five


**我们使用迭代发现，它返回了两类数据，所分隔的数据名称，和所分隔的数据本身，这个很像一个字典，我们在这个时候，也会选择将他转化为字典**

In [16]:
dic = dict(list(frame.groupby(['key1','key2'])))
print(dic)

{('a', 'one'):   key1 key2  data1     data2
0    a  one      0 -0.187529, ('a', 'three'):   key1   key2  data1     data2
2    a  three      2 -0.309963, ('g', 'five'):   key1  key2  data1     data2
4    g  five      4 -0.887752, ('s', 'four'):   key1  key2  data1     data2
3    s  four      3 -1.154808, ('s', 'two'):   key1 key2  data1     data2
1    s  two      1 -1.621045}


### 选取一列或一列的子集

In [17]:
#看例子
print(frame.groupby(['key1','key2'])['data1'].mean())

key1  key2 
a     one      0
      three    2
g     five     4
s     four     3
      two      1
Name: data1, dtype: int64


**从上面的例子可以看出，这样的逻辑是更清楚的，frame.groupby:表示表格型数据被分隔，frame.groupby([]):找到了被分割的列，frame.groupby([])[[]]:表示所运算的数据**

**上面的表示方法是前面:frame[].groupby([])的语法糖**

### 通过字典和Series来分组

**运用场景：你的数据的命名没有什么区分度和相关性，但是数据本身是有区分度和相关性的，那你就面临着给不同轴的重新命名的**

In [18]:
frame = pd.DataFrame({'key1':[4,5,6,7,8],'key2':[2,3,4,5,2],'data1':range(5),'data2':np.random.randn(5)})

In [19]:
print(frame)

   key1  key2  data1     data2
0     4     2      0  1.302274
1     5     3      1  0.450506
2     6     4      2  0.680666
3     7     5      3 -0.347008
4     8     2      4 -0.827686


In [20]:
state = {'key1':'hebei','key2':'hebei','data1':'handan','data2':'zibo'}

In [21]:
test = frame.groupby(state,axis=1)

In [22]:
test.mean()

,handan,hebei,zibo
0,0.0,3.0,1.302274
1,1.0,4.0,0.450506
2,2.0,5.0,0.680666
3,3.0,6.0,-0.347008
4,4.0,5.0,-0.827686


**如果是传入Series也可以达到相同的效果**

In [23]:
mapping = pd.Series(['hebi','hebi','handn','zibo'],index=['key1','key2','data1','data2'])

In [24]:
mapping

key1      hebi
key2      hebi
data1    handn
data2     zibo
dtype: object

In [25]:
test = frame.groupby(mapping,axis=1)

In [26]:
test.mean()

,handn,hebi,zibo
0,0.0,3.0,1.302274
1,1.0,4.0,0.450506
2,2.0,5.0,0.680666
3,3.0,6.0,-0.347008
4,4.0,5.0,-0.827686


### 通过函数进行分组

**首先我们要再次强调我们分组的条件是什么。分组的中的纲领是，根据我们的意愿来分组，如果你的默认的分组方式不能满足你的需求，那么你需要制定一些规则：前面我们学的规则有（重命名，不同列的选取）。**

**使用函数的方法可以大大的加强分组的灵活性**

In [27]:
frame = pd.DataFrame(np.ceil(np.random.uniform(1,29,(5,5))),index=[1,2,3,1,2],columns=['a','s','d','f','a'])   #字典方式录入和以column\index方式录入不同
print(frame)

      a     s     d     f     a
1   6.0  13.0  27.0  24.0   2.0
2  16.0  12.0  29.0   5.0  14.0
3  19.0  18.0   7.0  14.0   5.0
1  19.0  28.0   6.0  13.0   4.0
2   5.0  10.0   6.0  29.0  15.0


In [28]:
frame.groupby(frame.columns,axis=1).sum()

,a,d,f,s
1,8.0,27.0,24.0,13.0
2,30.0,29.0,5.0,12.0
3,24.0,7.0,14.0,18.0
1,23.0,6.0,13.0,28.0
2,20.0,6.0,29.0,10.0


**解释一下上面的过程，我们使用了len函数，来统计columns上的字符长度，然后对于相同长度的string做了统计，然后计算了mean()**

### 根据索引级别分组

**这个是好理解的，在层次化索引下，我们的数据也呈现出规律，通过不同的索引级别，我们可以做出区分**

In [29]:
frame = pd.DataFrame(np.ceil(np.random.uniform(1,100000,(4,4))),columns=[['tianzhu','tianzhu','wuwei','wuwei'],['bwjb','hebw','lala','mwmw']])
print(frame)

   tianzhu             wuwei         
      bwjb     hebw     lala     mwmw
0  48526.0  53677.0  35155.0  13314.0
1  37452.0  81756.0  74257.0  61304.0
2  75895.0  23028.0  87861.0  27419.0
3  85459.0  16474.0  47158.0  61234.0


In [30]:
frame.columns.names=['name','iii']
print(frame)

name  tianzhu             wuwei         
iii      bwjb     hebw     lala     mwmw
0     48526.0  53677.0  35155.0  13314.0
1     37452.0  81756.0  74257.0  61304.0
2     75895.0  23028.0  87861.0  27419.0
3     85459.0  16474.0  47158.0  61234.0


In [31]:
frame.groupby(level='name',axis=1).mean()

name,tianzhu,wuwei
0,51101.5,24234.5
1,59604.0,67780.5
2,49461.5,57640.0
3,50966.5,54196.0


# 数据集合

**我们前面已经使用了mean()函数，对于这个函数，我们可将离散的数据，做聚合运算，然后返回计算结果**

In [32]:
#方式1
frame = pd.DataFrame(np.ceil(np.random.uniform(1,29,(5,5))),index=[1,2,3,1,2],columns=['a','s','d','f','a'])   #字典方式录入和以column\index方式录入不同
print(frame)

      a     s     d     f     a
1  25.0  13.0   9.0  10.0  20.0
2  16.0   6.0   9.0   5.0  14.0
3  13.0  25.0  13.0   5.0   4.0
1  25.0   6.0  15.0   3.0  10.0
2  14.0  16.0  10.0   5.0   3.0


In [33]:
frame.groupby(frame.columns,axis=1).mean()

,a,d,f,s
1,22.5,9.0,10.0,13.0
2,15.0,9.0,5.0,6.0
3,8.5,13.0,5.0,25.0
1,17.5,15.0,3.0,6.0
2,8.5,10.0,5.0,16.0


**对于合并的方法，我们也可以使用函数的方法来将数据整合，将函数传入agg或者aggregate()中**

In [34]:
def top_to_top(arr):
    return arr.max()-arr.min()

In [35]:
frame = pd.DataFrame(np.ceil(np.random.uniform(1,29,(5,5))),index=[1,2,3,1,2],columns=['a','s','d','f','a'])   #字典方式录入和以column\index方式录入不同
print(frame)

      a     s     d     f     a
1  20.0   8.0   2.0  25.0  26.0
2  11.0  18.0  28.0  14.0  26.0
3   5.0   4.0  14.0  29.0  13.0
1  14.0  21.0  25.0  14.0   2.0
2  15.0  28.0  14.0  12.0  24.0


In [36]:
frame.groupby(frame.index).agg(top_to_top)

,a,s,d,f,a
1,6.0,13.0,23.0,11.0,24.0
2,4.0,10.0,14.0,2.0,2.0
3,0.0,0.0,0.0,0.0,0.0


In [37]:
#还可以使用describe
frame.groupby(frame.columns).describe()

a                                                     s        ...  \
  count  mean       std   min    25%   50%    75%   max count  mean  ...   
a   2.0  17.5  3.535534  15.0  16.25  17.5  18.75  20.0   2.0  18.0  ...   
d   1.0   5.0       NaN   5.0   5.00   5.0   5.00   5.0   1.0   4.0  ...   
f   1.0  14.0       NaN  14.0  14.00  14.0  14.00  14.0   1.0  21.0  ...   
s   1.0  11.0       NaN  11.0  11.00  11.0  11.00  11.0   1.0  18.0  ...   

       f           a                                                
     75%   max count  mean       std   min   25%   50%   75%   max  
a  21.75  25.0   2.0  25.0  1.414214  24.0  24.5  25.0  25.5  26.0  
d  29.00  29.0   1.0  13.0       NaN  13.0  13.0  13.0  13.0  13.0  
f  14.00  14.0   1.0   2.0       NaN   2.0   2.0   2.0   2.0   2.0  
s  14.00  14.0   1.0  26.0       NaN  26.0  26.0  26.0  26.0  26.0  

[4 rows x 40 columns]

### 面向列的多函数应用

**我们前面的聚合运算可以发现，我们单次的使用了聚合函数。但是如果，我多次的使用不同的函数，那么我要怎么做呢？**

In [38]:
tips = pd.read_csv('E:/Datawhale数据分析/PythonForDataAnalysis-master/PythonForDataAnalysis-master/ch08/tips.csv')

In [39]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [40]:
tips['tips_pct'] = tips['tip']/tips['total_bill']

In [41]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tips_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [42]:
groups = tips.groupby(['day','smoker'])

In [43]:
groups['tips_pct'].mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tips_pct, dtype: float64

In [44]:
groups['tips_pct'].agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tips_pct, dtype: float64

In [45]:
print(groups['tips_pct'].agg(['mean'])) #注意与上面的区别

                 mean
day  smoker          
Fri  No      0.151650
     Yes     0.174783
Sat  No      0.158048
     Yes     0.147906
Sun  No      0.160113
     Yes     0.187250
Thur No      0.160298
     Yes     0.163863


In [46]:
tips.groupby(['day','smoker']).agg(['mean'])

total_bill       tip      size  tips_pct
                  mean      mean      mean      mean
day  smoker                                         
Fri  No      18.420000  2.812500  2.250000  0.151650
     Yes     16.813333  2.714000  2.066667  0.174783
Sat  No      19.661778  3.102889  2.555556  0.158048
     Yes     21.276667  2.875476  2.476190  0.147906
Sun  No      20.506667  3.167895  2.929825  0.160113
     Yes     24.120000  3.516842  2.578947  0.187250
Thur No      17.113111  2.673778  2.488889  0.160298
     Yes     19.190588  3.030000  2.352941  0.163863

**上面的操作大概可以分为两类：1.将数据分隔之后，不断的调用其中某块数据做聚合运算 2.在调用聚合运算时，调用的方法是(['调用方法'])，可以显示在运算之后的数据表中**

**多种的聚合运算：**

In [47]:
grouped.agg(['mean','sum','std',top_to_top])  #top_to_top函数上面已经调用过

data1                              data2                               
      mean sum       std top_to_top      mean       sum       std top_to_top
key1                                                                        
a        1   2  1.414214          2 -0.248746 -0.497492  0.086574   0.122435
g        4   4       NaN          0 -0.887752 -0.887752       NaN   0.000000
s        2   4  1.414214          2 -1.387926 -2.775853  0.329680   0.466238

### 以‘没有行索引’的形式返回聚合数据

**这个可以使用reset_index()来完成**

In [48]:
tips.groupby(['day','smoker'],as_index=False).agg(['mean'],as_index=False).reset_index()

,day,smoker,total_bill,tip,size,tips_pct
,,,mean,mean,mean,mean
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 关于apply()的一般运用

**我们知道表格型数据本身我们要做复杂的运算时，我们需要将数据分隔--运算--聚合。对于pandas提供的运算中apply运用的十分广泛，它可以对多列的数据做运算（这个agg不能完成），然后将分隔并已经运算完成的数据合并起来**

In [49]:
tips = pd.read_csv('E:/Datawhale数据分析/PythonForDataAnalysis-master/PythonForDataAnalysis-master/ch08/tips.csv')
print(tips.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [50]:
tips['pre_tip'] = tips['tip'] / tips['total_bill']

In [51]:
print(tips.head())

   total_bill   tip     sex smoker  day    time  size   pre_tip
0       16.99  1.01  Female     No  Sun  Dinner     2  0.059447
1       10.34  1.66    Male     No  Sun  Dinner     3  0.160542
2       21.01  3.50    Male     No  Sun  Dinner     3  0.166587
3       23.68  3.31    Male     No  Sun  Dinner     2  0.139780
4       24.59  3.61  Female     No  Sun  Dinner     4  0.146808


**去pre_tip为参考系，取前五名**

In [52]:
grouped  = tips.groupby(['smoker','sex'])

In [53]:
def pre_sort (frame,n=5,column = 'pre_tip'):
    return tips.sort_values(by=column)[-n:]

In [54]:
grouped.apply(pre_sort)

total_bill   tip     sex smoker  day    time  size  \
smoker sex                                                              
No     Female 183       23.17  6.50    Male    Yes  Sun  Dinner     4   
              232       11.61  3.39    Male     No  Sat  Dinner     2   
              67         3.07  1.00  Female    Yes  Sat  Dinner     1   
              178        9.60  4.00  Female    Yes  Sun  Dinner     2   
              172        7.25  5.15    Male    Yes  Sun  Dinner     2   
       Male   183       23.17  6.50    Male    Yes  Sun  Dinner     4   
              232       11.61  3.39    Male     No  Sat  Dinner     2   
              67         3.07  1.00  Female    Yes  Sat  Dinner     1   
              178        9.60  4.00  Female    Yes  Sun  Dinner     2   
              172        7.25  5.15    Male    Yes  Sun  Dinner     2   
Yes    Female 183       23.17  6.50    Male    Yes  Sun  Dinner     4   
              232       11.61  3.39    Male     No  Sat  Dinner     2   
              67         3.07  1.00  Female    Yes  Sat  Dinner     1   
              178        9.60  4.00  Female    Yes  Sun  Dinner     2   
              172        7.25  5.15    Male    Yes  Sun  Dinner     2   
       Male   183       23.17  6.50    Male    Yes  Sun  Dinner     4   
              232       11.61  3.39    Male     No  Sat  Dinner     2   
              67         3.07  1.00  Female    Yes  Sat  Dinner     1   
              178        9.60  4.00  Female    Yes  Sun  Dinner     2   
              172        7.25  5.15    Male    Yes  Sun  Dinner     2   

                    pre_tip  
smoker sex                   
No     Female 183  0.280535  
              232  0.291990  
              67   0.325733  
              178  0.416667  
              172  0.710345  
       Male   183  0.280535  
              232  0.291990  
              67   0.325733  
              178  0.416667  
              172  0.710345  
Yes    Female 183  0.280535  
              232  0.291990  
              67   0.325733  
              178  0.416667  
              172  0.710345  
       Male   183  0.280535  
              232  0.291990  
              67   0.325733  
              178  0.416667  
              172  0.710345

**如果你的函数的需要传参，你用apply()时，把所需的参数传入的后边就好了**

In [55]:
grouped.apply(pre_sort,n=4,column='tip')

total_bill    tip   sex smoker  day    time  size   pre_tip
smoker sex                                                                    
No     Female 59        48.27   6.73  Male     No  Sat  Dinner     4  0.139424
              23        39.42   7.58  Male     No  Sat  Dinner     4  0.192288
              212       48.33   9.00  Male     No  Sat  Dinner     4  0.186220
              170       50.81  10.00  Male    Yes  Sat  Dinner     3  0.196812
       Male   59        48.27   6.73  Male     No  Sat  Dinner     4  0.139424
              23        39.42   7.58  Male     No  Sat  Dinner     4  0.192288
              212       48.33   9.00  Male     No  Sat  Dinner     4  0.186220
              170       50.81  10.00  Male    Yes  Sat  Dinner     3  0.196812
Yes    Female 59        48.27   6.73  Male     No  Sat  Dinner     4  0.139424
              23        39.42   7.58  Male     No  Sat  Dinner     4  0.192288
              212       48.33   9.00  Male     No  Sat  Dinner     4  0.186220
              170       50.81  10.00  Male    Yes  Sat  Dinner     3  0.196812
       Male   59        48.27   6.73  Male     No  Sat  Dinner     4  0.139424
              23        39.42   7.58  Male     No  Sat  Dinner     4  0.192288
              212       48.33   9.00  Male     No  Sat  Dinner     4  0.186220
              170       50.81  10.00  Male    Yes  Sat  Dinner     3  0.196812